In [1]:
import pandas as pd

PATH = "../빅콘테스트2020/DATA/2020빅콘테스트 데이터분석분야-퓨쳐스리그_스포츠투아이_제공데이터(.CSV)_시즌별, 시트별 구분/"

pitcher_2016 = pd.read_csv(PATH + '2020빅콘테스트_스포츠투아이_제공데이터_개인투수_2016.csv', encoding='utf-8-sig', engine='python')
pitcher_2017 = pd.read_csv(PATH + '2020빅콘테스트_스포츠투아이_제공데이터_개인투수_2017.csv', encoding='utf-8-sig', engine='python')
pitcher_2018 = pd.read_csv(PATH + '2020빅콘테스트_스포츠투아이_제공데이터_개인투수_2018.csv', encoding='utf-8-sig', engine='python')
pitcher_2019 = pd.read_csv(PATH + '2020빅콘테스트_스포츠투아이_제공데이터_개인투수_2019.csv', encoding='utf-8-sig', engine='python')
pitcher_2020 = pd.read_csv(PATH + '2020빅콘테스트_스포츠투아이_제공데이터_개인투수_2020.csv', encoding='utf-8-sig', engine='python')
pitcher_list = [pitcher_2016, pitcher_2017, pitcher_2018, pitcher_2019, pitcher_2020]
pitcher = pd.concat(pitcher_list)

In [2]:
new_columns = "게임키 일자 팀코드 상대팀코드 더블헤더코드 초말 선수코드 선발 구원 완투 종료 결과 홀드 이닝*3 투구수 타자 타수 안타 2루타 3루타 홈런 도루 도루실패 희타 희비 4구 고4 사구 삼진 병살타 폭투 보크 실책 실점 자책점 득점권WHIP 2점차이하WHIP 345번타자WHIP"
new_columns = new_columns.split(' ')
pitcher.columns = new_columns

In [3]:
pitcher['일자'] = pd.to_datetime(pitcher['일자'],format='%Y%m%d', errors='raise')
pitcher['시즌'] = pitcher['일자'].dt.year

In [4]:
team_pitcher_2016 = pd.read_csv(PATH + '2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2016.csv', encoding='utf-8-sig', engine='python')
team_pitcher_2017 = pd.read_csv(PATH + '2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2017.csv', encoding='utf-8-sig', engine='python')
team_pitcher_2018 = pd.read_csv(PATH + '2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2018.csv', encoding='utf-8-sig', engine='python')
team_pitcher_2019 = pd.read_csv(PATH + '2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2019.csv', encoding='utf-8-sig', engine='python')
team_pitcher_2020 = pd.read_csv(PATH + '2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2020.csv', encoding='utf-8-sig', engine='python')
team_pitcher_list = [team_pitcher_2016, team_pitcher_2017, team_pitcher_2018, team_pitcher_2019, team_pitcher_2020]
team_pitcher = pd.concat(team_pitcher_list)

In [5]:
new_columns = "게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,완투,결과,홀드,이닝*3,투구수,타자,타수,안타,2루타,3루타,홈런,도루,도루실패,희타,희비,4구,고4,사구,삼진,병살타,폭투,보크,실책,실점,자책점,득점권WHIP,2점차이하WHIP,345번타자WHIP"
new_columns = new_columns.split(',')

In [6]:
team_pitcher.columns = new_columns

In [7]:
team_pitcher['일자'] = pd.to_datetime(team_pitcher['일자'],format='%Y%m%d', errors='raise')
team_pitcher['시즌'] = team_pitcher['일자'].dt.year

In [8]:
def createERA_sr(row, sr):
    previous = pitcher[(pitcher['팀코드'] == row['팀코드']) & (pitcher['선발'] == sr)].copy()
    previous = previous[(previous['시즌'] == row['시즌']) &(previous['일자'] < row['일자'])]
    if sr==1:
        if len(previous) == 0:
            row['선발_평균자책점'] = 0
            return row
        up = previous['자책점'].sum()
        down = previous['이닝*3'].sum() / 3
        if down == 0:
            row['선발_평균자책점'] = 0
            return row
        row['선발_평균자책점'] = (up * 9) / down
        return row
    if sr==0:
        if len(previous) == 0:
            row['구원_평균자책점'] = 0
            return row
        up = previous['자책점'].sum()
        down = previous['이닝*3'].sum() / 3
        if down == 0:
            row['구원_평균자책점'] = 0
            return row
        row['구원_평균자책점'] = (up * 9) / down
        return row

In [9]:
def createRA_sr(row,sr):
    previous = pitcher[(pitcher['팀코드'] == row['팀코드']) &(pitcher['선발'] == sr)].copy()
    previous = previous[(previous['시즌'] == row['시즌']) &(previous['일자'] < row['일자'])]
    if sr==1:
        if len(previous) == 0:
            row['선발_RA/9'] = 0
            return row
        up = previous['실점'].sum()
        down = previous['이닝*3'].sum() / 3
        if down == 0:
            row['선발_RA/9'] = 0
            return row
        row['선발_RA/9'] = (up * 9) / down
        return row
    if sr==0:
        if len(previous) == 0:
            row['구원_RA/9'] = 0
            return row
        up = previous['실점'].sum()
        down = previous['이닝*3'].sum() / 3
        if down == 0:
            row['구원_RA/9'] = 0
            return row
        row['구원_RA/9'] = (up * 9) / down
        return row

In [10]:
def createERC_sr(row, sr):
    previous = pitcher[(pitcher['팀코드'] == row['팀코드']) & (pitcher['선발'] == sr)].copy()
    previous = previous[(previous['시즌'] == row['시즌']) &(previous['일자'] < row['일자'])]
    if sr == 1:
        if len(previous) == 0:
            row['선발_ERC'] = 0
            return row
        previous_sum = previous.sum()
        PTB = 0.89 * (1.255 * (previous_sum['안타'] - previous_sum['홈런']) + 4 * previous_sum['홈런']) + 0.475 * (previous_sum['4구'] + previous_sum['사구'] - previous_sum['고4'])
        down = (previous_sum['타자'] * previous_sum['이닝*3'] / 3)
        if down == 0:
            row['선발_ERC'] = 0
            return row
        ERC = 9* ( ((previous_sum['안타'] + previous_sum['4구'] + previous_sum['사구']) * PTB) / down ) - 0.56
        if ERC < 2.24:
            ERC = 9* ( ((previous_sum['안타'] + previous_sum['4구'] + previous_sum['사구']) * PTB) / down ) * 0.75
        row['선발_ERC'] = ERC
        return row
    if sr == 0:
        if len(previous) == 0:
            row['구원_ERC'] = 0
            return row
        previous_sum = previous.sum()
        PTB = 0.89 * (1.255 * (previous_sum['안타'] - previous_sum['홈런']) + 4 * previous_sum['홈런']) + 0.475 * (previous_sum['4구'] + previous_sum['사구'] - previous_sum['고4'])
        down = (previous_sum['타자'] * previous_sum['이닝*3'] / 3)
        if down == 0:
            row['구원_ERC'] = 0
            return row
        ERC = 9* ( ((previous_sum['안타'] + previous_sum['4구'] + previous_sum['사구']) * PTB) / down ) - 0.56
        if ERC < 2.24:
            ERC = 9* ( ((previous_sum['안타'] + previous_sum['4구'] + previous_sum['사구']) * PTB) / down ) * 0.75
        row['구원_ERC'] = ERC
        return row

In [11]:
def createFIP_sr(row, sr):
    C_dic = {2016: 3.7732111715557406,
     2017: 3.7201314451138408,
     2018: 3.810512298292339,
     2019: 3.3880686560934894,
     2020: 3.596723044397463}
    previous = pitcher[(pitcher['시즌'] == row['시즌']) & (pitcher['팀코드'] == row['팀코드']) & (pitcher['일자'] < row['일자']) & (pitcher['선발'] == sr)]
    if sr == 1:
        if len(previous) == 0:
            row['선발_FIP'] = 0
            return row
        previous_sum = previous.sum()
        down = (previous_sum['이닝*3'] / 3)
        if down == 0 :
            row['선발_FIP'] = 0
            return row
        FIP = (( 13 * previous_sum['홈런'] + 3 * (previous_sum['4구'] + previous_sum['사구'] - previous_sum['고4']) - 2 * previous_sum['삼진']) /down) + C_dic[row['시즌']]
        row['선발_FIP'] = FIP
        return row
    if sr == 0:
        if len(previous) == 0:
            row['구원_FIP'] = 0
            return row
        previous_sum = previous.sum()
        down = (previous_sum['이닝*3'] / 3)
        if down == 0 :
            row['구원_FIP'] = 0
            return row
        FIP = (( 13 * previous_sum['홈런'] + 3 * (previous_sum['4구'] + previous_sum['사구'] - previous_sum['고4']) - 2 * previous_sum['삼진']) /down) + C_dic[row['시즌']]
        row['구원_FIP'] = FIP
        return row

def createLOB_sr(row, sr):
    previous = pitcher[(pitcher['시즌'] == row['시즌']) & (pitcher['팀코드'] == row['팀코드']) & (pitcher['일자'] < row['일자']) & (pitcher['선발'] == sr)]
    if sr == 1:
        if len(previous) == 0:
            row['선발_LOB'] = 0
            return row
        previous_sum = previous.sum()
        up = previous_sum['안타'] + previous_sum['4구'] + previous_sum['사구'] - previous_sum['실점']
        down = previous_sum['안타'] + previous_sum['4구'] + previous_sum['사구'] - (1.4 * previous_sum['홈런'])
        if down == 0:
            row['선발_LOB'] = 0
            return row 
        row['선발_LOB'] = up / down
        return row
    if sr == 0:
        if len(previous) == 0:
            row['구원_LOB'] = 0
            return row
        previous_sum = previous.sum()
        up = previous_sum['안타'] + previous_sum['4구'] + previous_sum['사구'] - previous_sum['실점']
        down = previous_sum['안타'] + previous_sum['4구'] + previous_sum['사구'] - (1.4 * previous_sum['홈런'])
        if down == 0:
            row['구원_LOB'] = 0
            return row 
        row['구원_LOB'] = up / down
        return row

In [12]:
def createK_HR_sr(row,sr):
    previous = pitcher[(pitcher['시즌'] == row['시즌']) & (pitcher['팀코드'] == row['팀코드']) & (pitcher['일자'] < row['일자']) & (pitcher['선발']==sr)]
    if sr==1:
        if len(previous) == 0:
            row['선발_홈런율'] = 0
            row['선발_삼진율'] = 0
            return row
        previous_sum = previous.sum()
        up_HR = previous_sum['홈런'] 
        up_k = previous_sum['삼진']
        down = previous_sum['타자'] - previous_sum['고4']
        if down == 0:
            row['선발_홈런율'] = 0
            row['선발_삼진율'] = 0
            return row
        row['선발_홈런율'] = up_HR / down
        row['선발_삼진율'] = up_k / down
        return row
    if sr==0:
        if len(previous) == 0:
            row['구원_홈런율'] = 0
            row['구원_삼진율'] = 0
            return row
        previous_sum = previous.sum()
        up_HR = previous_sum['홈런'] 
        up_k = previous_sum['삼진']
        down = previous_sum['타자'] - previous_sum['고4']
        if down == 0:
            row['구원_홈런율'] = 0
            row['구원_삼진율'] = 0
            return row
        row['구원_홈런율'] = up_HR / down
        row['구원_삼진율'] = up_k / down
        return row

In [13]:
def create9_sr(row,sr):
    previous = pitcher[(pitcher['시즌'] == row['시즌']) & (pitcher['팀코드'] == row['팀코드']) & (pitcher['일자'] < row['일자']) & (pitcher['선발'] == sr)]
    if sr==1:
        previous = previous[previous['선발']==1]
        if len(previous) == 0:
            row['선발_K9'] = 0
            row['선발_BB9'] = 0
            row['선발_HR9'] = 0
            row['선발_KBB'] = 0
        previous_sum = previous.sum()
        up_K = previous_sum['삼진'] 
        up_BB = previous_sum['4구'] - previous_sum['고4']
        up_HR = previous_sum['홈런']
        down = previous_sum['이닝*3'] / 3
        if down == 0 or up_BB == 0 :
            row['선발_K9'] = 0
            row['선발_BB9'] = 0
            row['선발_HR9'] = 0
            row['선발_KBB'] = 0
            return row
        row['선발_K9'] = up_K / down * 9
        row['선발_BB9'] = up_BB / down * 9
        row['선발_HR9'] = up_HR / down * 9
        row['선발_KBB'] = up_K / up_BB
        return row
    if sr==0:
        previous = previous[previous['선발']==0]
        if len(previous) == 0:
            row['구원_K9'] = 0
            row['구원_BB9'] = 0
            row['구원_HR9'] = 0
            row['구원_KBB'] = 0
        previous_sum = previous.sum()
        up_K = previous_sum['삼진'] 
        up_BB = previous_sum['4구'] - previous_sum['고4']
        up_HR = previous_sum['홈런']
        down = previous_sum['이닝*3'] / 3
        if down == 0 or up_BB == 0 :
            row['구원_K9'] = 0
            row['구원_BB9'] = 0
            row['구원_HR9'] = 0
            row['구원_KBB'] = 0
            return row
        row['구원_K9'] = up_K / down * 9
        row['구원_BB9'] = up_BB / down * 9
        row['구원_HR9'] = up_HR / down * 9
        row['구원_KBB'] = up_K / up_BB
        return row

In [14]:
team_pitcher_sr1 = team_pitcher.apply(createERA_sr,sr=1, axis=1)
team_pitcher_sr2 = team_pitcher.apply(createERA_sr,sr=0, axis=1)
team_pitcher_RA_sr1 = team_pitcher.apply(createRA_sr,sr=1, axis=1)
team_pitcher_RA_sr2 = team_pitcher.apply(createRA_sr,sr=0, axis=1)
team_pitcher_ERC_sr1 = team_pitcher.apply(createERC_sr,sr=1, axis=1)
team_pitcher_ERC_sr2 = team_pitcher.apply(createERC_sr,sr=0, axis=1)
team_pitcher_FIP_sr1 = team_pitcher.apply(createFIP_sr,sr=1, axis=1)
team_pitcher_FIP_sr2 = team_pitcher.apply(createFIP_sr,sr=0, axis=1)
team_pitcher_LOB_sr1 = team_pitcher.apply(createLOB_sr,sr=1, axis=1)
team_pitcher_LOB_sr2 = team_pitcher.apply(createLOB_sr,sr=0, axis=1)
team_pitcher_K_HR_sr1 = team_pitcher.apply(createK_HR_sr,sr=1, axis=1)
team_pitcher_K_HR_sr2 = team_pitcher.apply(createK_HR_sr,sr=0, axis=1)
team_pitcher_9_sr1 = team_pitcher.apply(create9_sr,sr=1, axis=1)
team_pitcher_9_sr2 = team_pitcher.apply(create9_sr,sr=0, axis=1)

In [17]:
team_pitcher_sr = pd.concat([team_pitcher_sr1,team_pitcher_sr2, team_pitcher_RA_sr1,team_pitcher_RA_sr2, team_pitcher_ERC_sr1, team_pitcher_ERC_sr2, 
                          team_pitcher_FIP_sr1, team_pitcher_FIP_sr2, team_pitcher_LOB_sr1, team_pitcher_LOB_sr2, team_pitcher_K_HR_sr1, team_pitcher_K_HR_sr2
                          team_pitcher_9_sr1,team_pitcher_9_sr2], axis =1 ,keys = ['게임키','일자','팀코드','상대팀코드','더블헤더코드','초말','완투','결과','홀드','이닝*3','투구수','타자','타수',
                                                                       '안타','2루타','3루타','홈런','도루','도루실패','희타','희비','4구','고4','사구','삼진','병살타','폭투','보크','실책','실점',
                                                                       '자책점','득점권WHIP','2점차이하WHIP','345번타자WHIP'])

SyntaxError: invalid syntax (<ipython-input-17-632cc2fd836b>, line 3)

In [27]:
def merge_(tp_list):
    df1 = tp_list[0]
    df2 = tp_list[1]
    team_pitcher_sr = pd.merge(df1,df2, on = ['게임키','일자','팀코드','상대팀코드','더블헤더코드','초말','완투','결과','홀드','이닝*3','투구수','타자','타수',
                                                '안타','2루타','3루타','홈런','도루','도루실패','희타','희비','4구','고4','사구','삼진','병살타','폭투','보크','실책','실점',
                                                '자책점','득점권WHIP','2점차이하WHIP','345번타자WHIP','시즌'])
    for i in range(len(tp_list)-3):
        team_pitcher_sr = pd.merge(team_pitcher_sr,tp_list[i+2], on = ['게임키','일자','팀코드','상대팀코드','더블헤더코드','초말','완투','결과','홀드','이닝*3','투구수','타자','타수',
                                                '안타','2루타','3루타','홈런','도루','도루실패','희타','희비','4구','고4','사구','삼진','병살타','폭투','보크','실책','실점',
                                                '자책점','득점권WHIP','2점차이하WHIP','345번타자WHIP','시즌'])
    return team_pitcher_sr
    

In [28]:
tp_list = [team_pitcher_sr1,team_pitcher_sr2, team_pitcher_RA_sr1,team_pitcher_RA_sr2, team_pitcher_ERC_sr1, team_pitcher_ERC_sr2, team_pitcher_FIP_sr1, team_pitcher_FIP_sr2, team_pitcher_LOB_sr1, team_pitcher_LOB_sr2, team_pitcher_K_HR_sr1, team_pitcher_K_HR_sr2, team_pitcher_9_sr1,team_pitcher_9_sr2]

team_pitcher_sr = merge_(tp_list)

In [29]:
team_pitcher_sr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6400 entries, 0 to 6399
Data columns (total 53 columns):
게임키           6400 non-null object
일자            6400 non-null datetime64[ns]
팀코드           6400 non-null object
상대팀코드         6400 non-null object
더블헤더코드        6400 non-null int64
초말            6400 non-null object
완투            6400 non-null int64
결과            6400 non-null object
홀드            6400 non-null int64
이닝*3          6400 non-null int64
투구수           6400 non-null int64
타자            6400 non-null int64
타수            6400 non-null int64
안타            6400 non-null int64
2루타           6400 non-null int64
3루타           6400 non-null int64
홈런            6400 non-null int64
도루            6400 non-null int64
도루실패          6400 non-null int64
희타            6400 non-null int64
희비            6400 non-null int64
4구            6400 non-null int64
고4            6400 non-null int64
사구            6400 non-null int64
삼진            6400 non-null int64
병살타           6400 non-null i

In [30]:
team_pitcher_sr.isnull().sum()

게임키           0
일자            0
팀코드           0
상대팀코드         0
더블헤더코드        0
초말            0
완투            0
결과            0
홀드            0
이닝*3          0
투구수           0
타자            0
타수            0
안타            0
2루타           0
3루타           0
홈런            0
도루            0
도루실패          0
희타            0
희비            0
4구            0
고4            0
사구            0
삼진            0
병살타           0
폭투            0
보크            0
실책            0
실점            0
자책점           0
득점권WHIP       0
2점차이하WHIP     0
345번타자WHIP    0
시즌            0
선발_평균자책점      0
구원_평균자책점      0
선발_RA/9       0
구원_RA/9       0
선발_ERC        0
구원_ERC        0
선발_FIP        0
구원_FIP        0
선발_LOB        0
구원_LOB        0
선발_홈런율        0
선발_삼진율        0
구원_홈런율        0
구원_삼진율        0
선발_K9         0
선발_BB9        0
선발_HR9        0
선발_KBB        0
dtype: int64